In [168]:
import numpy as np
import pandas as pd
import math


In [203]:
# Постоянные
PLAYERS = 5 #
GAMES = 10 #

SEED_MONEY = 100

WIN_RATE = 1.5
LOSS_RATE = 0.6

In [204]:
matr = np.zeros(GAMES * PLAYERS).reshape(PLAYERS, -1)
matr[:,0] = SEED_MONEY


In [205]:
for i in range(1, matr.shape[1]):
    for j in range(matr.shape[0]):
        matr[j,i] = matr[j, i-1] * WIN_RATE if np.random.randint(2) else matr[j, i-1] * LOSS_RATE

In [206]:
matr

array([[100.     ,  60.     ,  90.     , 135.     , 202.5    , 121.5    ,
        182.25   , 273.375  , 164.025  , 246.0375 ],
       [100.     , 150.     ,  90.     , 135.     , 202.5    , 121.5    ,
         72.9    , 109.35   , 164.025  ,  98.415  ],
       [100.     , 150.     , 225.     , 135.     ,  81.     ,  48.6    ,
         29.16   ,  17.496  ,  10.4976 ,   6.29856],
       [100.     ,  60.     ,  36.     ,  54.     ,  81.     ,  48.6    ,
         29.16   ,  43.74   ,  65.61   ,  98.415  ],
       [100.     , 150.     ,  90.     ,  54.     ,  32.4    ,  19.44   ,
         11.664  ,  17.496  ,  26.244  ,  15.7464 ]])

In [207]:

df = pd.DataFrame(data=matr)

In [208]:
# players = []
# steps = []
# money = []
d = []
rate = 1

In [209]:
for i in range(PLAYERS):
    x = SEED_MONEY
    for j in range(GAMES):
        d.append({"players": i, "steps": j, "money": x})

        if np.random.randint(2):
            x = x * (1-rate) + x * rate * WIN_RATE
        else:
            x = x * (1-rate) + x * rate * LOSS_RATE
        


In [210]:
df = pd.DataFrame(d)
df.T


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
players,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.000,...,4.0,4.0,4.0,4.0,4.00,4.00,4.00,4.00,4.00,4.000
steps,0.0,1.0,2.0,3.0,4.0,5.00,6.00,7.00,8.00,9.000,...,0.0,1.0,2.0,3.0,4.00,5.00,6.00,7.00,8.00,9.000
money,100.0,150.0,225.0,337.5,202.5,303.75,182.25,109.35,65.61,98.415,...,100.0,60.0,36.0,21.6,12.96,19.44,29.16,43.74,65.61,39.366


In [211]:
gdf = df.groupby('steps').sum('money')/PLAYERS
gdf.reset_index( inplace=True)
gdf

,steps,players,money
0,0,2.0,100.00000
1,1,2.0,114.00000
2,2,2.0,106.20000
3,3,2.0,136.62000
4,4,2.0,91.69200
5,5,2.0,108.37800
6,6,2.0,86.02200
7,7,2.0,69.98400
8,8,2.0,49.86360
9,9,2.0,58.26168


In [212]:
gdf['ideal'] = gdf['steps'] * 1.05

In [213]:
w_rate = 1 + (WIN_RATE-1) * rate
l_rate = 1 + (LOSS_RATE-1) * rate
l_rate

0.6

In [214]:
coef = 1 + (WIN_RATE + LOSS_RATE - 2) * rate / 2
q = math.sqrt((1 + (WIN_RATE-1) * rate) * (1 + (LOSS_RATE-1) * rate))
q

0.9486832980505138

In [215]:
gdf['ideal'] = SEED_MONEY * ((1 + (WIN_RATE + LOSS_RATE - 2) * rate / 2) ** gdf['steps'])
gdf['expected in time'] = SEED_MONEY * ((math.sqrt((1 + (WIN_RATE-1) * rate) * (1 + (LOSS_RATE-1) * rate))) ** gdf['steps'])

In [226]:
for i in range(11):
    rate = 0.1* i

    arifm_coef = 1 + (WIN_RATE + LOSS_RATE - 2) * rate / 2
    geom_coef = math.sqrt((1 + (WIN_RATE-1) * rate) * (1 + (LOSS_RATE-1) * rate))

    print(f'rate={rate}, arifm_coef={arifm_coef}, geom_coef={geom_coef}')

rate=0.0, arifm_coef=1.0, geom_coef=1.0
rate=0.1, arifm_coef=1.005, geom_coef=1.0039920318408906
rate=0.2, arifm_coef=1.01, geom_coef=1.0059821071967434
rate=0.30000000000000004, arifm_coef=1.0150000000000001, geom_coef=1.0059821071967434
rate=0.4, arifm_coef=1.02, geom_coef=1.0039920318408906
rate=0.5, arifm_coef=1.025, geom_coef=1.0
rate=0.6000000000000001, arifm_coef=1.03, geom_coef=0.9939818911831342
rate=0.7000000000000001, arifm_coef=1.0350000000000001, geom_coef=0.985900603509299
rate=0.8, arifm_coef=1.04, geom_coef=0.9757048734120374
rate=0.9, arifm_coef=1.045, geom_coef=0.9633275663033836
rate=1.0, arifm_coef=1.05, geom_coef=0.9486832980505138


In [228]:
rate=1
geom_coef = ((1 + (WIN_RATE-1) * rate) * (1 + (LOSS_RATE-1) * rate))
geom_coef

0.8999999999999999

In [234]:
dff = df[df['steps'] == max(df['steps'])].sort_values('money', ascending=[False])

In [236]:
dff[:3]

,players,steps,money
9,0,9,98.415
19,1,9,98.415
29,2,9,39.366


In [248]:
filter_players = df[df['steps'] == max(df['steps'])].sort_values('money', ascending=[False])[:3]['players']
filter_players

9     0
19    1
29    2
Name: players, dtype: int64

In [249]:
dff = df[df['players'].isin(filter_players)] 

In [255]:
df['players'].unique()

1

In [253]:
import random

x = random.shuffle(df['players'].unique())
x

In [257]:
index = np.random.choice(df['players'].unique(), 5, replace=False)
index

array([1, 2, 4, 3, 0])

In [258]:
dff = df[df['players'].isin(np.random.choice(df['players'].unique(), 5, replace=False))] 